In [12]:
import pyodbc
import pandas as pd
import csv

# Connect to the SQL Server database
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=Test;UID=TestUser;PWD=Ex@mUs3R110423')

# Create a cursor to execute SQL queries
cursor = cnxn.cursor()

# Open the attached file and parse the data
with open('BING_MultiDays.csv', 'r') as file:
    data = csv.reader(file)
    
    # Skip the first 9 rows due to file format
    for i in range(9):
        next(data)
    
    next(data) # skip the header row
    
    for row in data:
        # Insert the row into the SQL Server table
        cursor.execute("INSERT INTO [dbo].[ST_MultiDays](Gregorian_date, Customer, Account_number, Account_name, Account_status, Campaign_name, Campaign_status, Ad_group_ID, Ad_group, Ad_group_status, Ad_ID, Ad_description, Ad_distribution, Ad_status, Ad_title, Ad_type, Tracking_Template, Custom_Parameters, Final_Mobile_URL, Final_URL, Top_vs_other, Display_URL, Final_App_URL, Destination_URL, Device_type, Device_OS, Delivered_match_type, BidMatchType, Language, Network, Currency_code, Impressions, Clicks, Spend, Avg_position, Conversions, Assists) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row)

# Commit the changes to the database
cnxn.commit()

# Close the cursor and connection
cursor.close()
cnxn.close()

In [13]:
import pyodbc

# Establish a connection to the SQL Server database
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=Test;UID=TestUser;PWD=Ex@mUs3R110423')
# Create a cursor object to execute SQL statements
cursor = cnxn.cursor()

# Retrieve distinct account numbers and names from fact_table
cursor.execute("SELECT DISTINCT Account_Number, Account_Name, Account_Status FROM [dbo].[ST_MultiDays]")
account_rows = cursor.fetchall()

# Loop over the account rows and insert them into dim_account table
for row in account_rows:
    account_number = row[0]
    account_name = row[1]
    account_status = row[2]
    
    # Check if account already exists in dim_account table
    cursor.execute("SELECT COUNT(*) FROM DimAccount WHERE account_number=?", account_number)
    count = cursor.fetchone()[0]
    if count == 0:
        # Account does not exist, insert it into dim_account table
        cursor.execute("INSERT INTO DimAccount(Account_Number, Account_Name, Account_Status) VALUES (?, ?, ?)", 
            account_number, account_name, account_status)
    else:
        # Account already exists, update account_name in dim_account table
        cursor.execute("UPDATE DimAccount SET Account_Name=?, Account_Status=? WHERE account_number=?", 
            account_name, account_status, account_number)

# Commit the changes to the database
cnxn.commit()

# Close the cursor and connection objects
cursor.close()
cnxn.close()

In [14]:
import pyodbc

# Establish a connection to the SQL Server database
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=Test;UID=TestUser;PWD=Ex@mUs3R110423')
# Create a cursor object to execute SQL statements
cursor = cnxn.cursor()

# Retrieve distinct account numbers and names from fact_table
cursor.execute("SELECT DISTINCT TRIM(Campaign_Name) as Campaign_Name, TRIM(Campaign_Status) as Campaign_Status FROM [dbo].[ST_MultiDays] ORDER BY Campaign_Name ASC")
campaign_rows = cursor.fetchall()

# Loop over the account rows and insert them into dim_account table
for row in campaign_rows:
    campaign_name = row[0]
    campaign_status = row[1]
    
    # Check if account already exists in dim_account table
    cursor.execute("SELECT COUNT(*) FROM DimCampaign WHERE Campaign_Name=?", campaign_name)
    count = cursor.fetchone()[0]
    if count == 0:
        # Account does not exist, insert it into dim_account table
        cursor.execute("INSERT INTO DimCampaign(Campaign_Name, Campaign_Status) VALUES (?, ?)", 
            campaign_name, campaign_status)
    else:
        # Account already exists, update account_name in dim_account table
        cursor.execute("UPDATE DimCampaign SET Campaign_Name=?, Campaign_Status=? WHERE Campaign_Name=?", 
            campaign_name, campaign_status, campaign_name)

# Commit the changes to the database
cnxn.commit()

# Close the cursor and connection objects
cursor.close()
cnxn.close()